# Spotify Data Project

Authored by Nick Ivory
Version 1.0
5/18/25

## Project Overview - Exploratory Data Analysis with Spotify API Data

-----
I wanted to build a project that would help me learn and explore three subject areas:
- Proof of Concept 
- Using Polars' columnar data format to investigate performance improvements over Pandas
- Principal Component Analysis, to determine hidden insights into the underlying dataset

In lieu of actually accessing Spotify's open API, I used the following Kaggle dataset: [Spotify Tracks DB](https://www.kaggle.com/datasets/zaheenhamidani/ultimate-spotify-tracks-db/data)

I set out to answer the following questions:
> "How do I "


-----

## Install Dependencies

In [1]:
!pip install -r requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'


In [1]:
# Install dependencies as needed:
!pip install 'kagglehub[pandas-datasets]'
!pip install 'kagglehub[polars-datasets]'

You should consider upgrading via the '/Users/nicholasivory/dev/personal/spotify-data-project/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/nicholasivory/dev/personal/spotify-data-project/venv/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/nicholasivory/dev/personal/spotify-data-project/venv/bin/python3 -m pip install --upgrade pip' command.


## Import Library Dependencies

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import polars as pl

## Import Kaggle Dataset Using Pandas

In [3]:
# Set the path to the file you'd like to load
file_path = "SpotifyFeatures.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "zaheenhamidani/ultimate-spotify-tracks-db",
  file_path,
  # Provide any additional arguments like 
  # sql_query or pandas_kwargs. See the 
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

/var/folders/4v/4nbj6pg96h3c8v4mjpv3wd340000gn/T/ipykernel_26918/1261988224.py:5: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:    genre        artist_name                        track_name  \
0  Movie     Henri Salvador       C'est beau de faire un Show   
1  Movie  Martin & les fées  Perdu d'avance (par Gad Elmaleh)   
2  Movie    Joseph Williams    Don't Let Me Be Lonely Tonight   
3  Movie     Henri Salvador    Dis-moi Monsieur Gordon Cooper   
4  Movie       Fabien Nataf                         Ouverture   

                 track_id  popularity  acousticness  danceability  \
0  0BRjO6ga9RKCKjfDqeFgWV           0         0.611         0.389   
1  0BjC1NfoEOOusryehmNudP           1         0.246         0.590   
2  0CoSDzoNIKCRs124s9uTVy           3         0.952         0.663   
3  0Gc6TVm52BwZD07Ki6tIvf           0         0.703         0.240   
4  0IuslXpMROHdEPvSl1fTQK           4         0.950         0.331   

   duration_ms  energy  instrumentalness key  liveness  loudness   mode  \
0        99373   0.910             0.000  C#    0.3460    -1.828  Major   
1       137373   0.737     

## Import Kaggle Dataset Using Polars

In [ ]:
# Set the path to the file you'd like to load
file_path = "SpotifyFeatures.csv"

# Load the latest version
lf = kagglehub.load_dataset(
  KaggleDatasetAdapter.POLARS,
  "zaheenhamidani/ultimate-spotify-tracks-db",
  file_path,
  # Provide any additional arguments like
  # sql_query, polars_frame_type, or 
  # polars_kwargs.
  # See the documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpolars
)

print("First 5 records:", lf.collect().head())